In [2]:
# 安装必要的库
%pip install rapidocr_onnxruntime -i https://pypi.tuna.tsinghua.edu.cn/simple
%pip install "unstructured[all-docs]" -i https://pypi.tuna.tsinghua.edu.cn/simple
%pip install pyMuPDF -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.document_loaders import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("/workspaces/LLMEasyProject/database/knowledge_db/临床流行病学.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pages = loader.load()

In [6]:
print(f"载入后的变量类型为：{type(pages)}，",  f"该 PDF 一共包含 {len(pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 353 页


In [7]:
page = pages[50]
print(f"每一个元素的类型：{type(page)}.", 
    f"该文档的描述性数据：{page.metadata}", 
    f"查看该文档的内容:\n{page.page_content[0:100]}", 
    sep="\n------\n")


每一个元素的类型：<class 'langchain.schema.document.Document'>.
------
该文档的描述性数据：{'source': '/workspaces/LLMEasyProject/database/knowledge_db/临床流行病学.pdf', 'file_path': '/workspaces/LLMEasyProject/database/knowledge_db/临床流行病学.pdf', 'page': 50, 'total_pages': 353, 'format': 'PDF 1.5', 'title': '临床流行病学  八年制  配增值', 'author': '詹思延主编', 'subject': '', 'keywords': '', 'creator': 'Pdg2Pic', 'producer': 'FreePic2Pdf_Lib - v3.02', 'creationDate': "D:20160422180607+08'00'", 'modDate': "D:20160422180607+08'00'", 'trapped': ''}
------
查看该文档的内容:

